In [ ]:
print("OK!")

In [ ]:
pip install -U langchain-community

In [ ]:
from pinecone import Pinecone
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from tqdm import tqdm

In [ ]:
from langchain.vectorstores import Pinecone as LangchainPinecone
from pinecone import Pinecone, ServerlessSpec

In [ ]:
pip install pinecone

In [ ]:
from pinecone import Pinecone

# Initialize Pinecone
pc = Pinecone(api_key="e353333b-b041-4e67-888c-ea37f8b89dc5")

# Test the connection
print(pc.list_indexes().names())

In [ ]:
#PINECONE_API_KEY = "e353333b-b041-4e67-888c-ea37f8b89dc5"
#PINECONE_API_ENV = "us-east-1"

In [ ]:
#Extract data from the PDF
def load_pdf(data):
    print("Function load_pdf is running")
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents


In [ ]:
extracted_data = load_pdf("D:\End-to-end-Chatbot-using-Llama2\data/")

In [ ]:
# extracted_data

In [ ]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

In [ ]:
# text_chunks

In [ ]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()

In [ ]:
embeddings

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

In [ ]:
# query_result

In [ ]:
pip install tqdm

In [ ]:
# Pinecone index creation and data insertion
index_name = "chatbot"

# Check if index exists, create if it doesn't
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimension of all-MiniLM-L6-v2 embeddings
        metric="cosine"
    )

# Get the index
index = pc.Index(index_name)

# Debug: Check if text_chunks exists and is not empty
if 'text_chunks' not in globals():
    print("Error: text_chunks is not defined. Make sure you've run the previous cells.")
elif not text_chunks:
    print("Error: text_chunks is empty. Check if the PDF loading was successful.")
else:
    print(f"Number of text chunks: {len(text_chunks)}")

    # Create embeddings for text chunks
    texts = [t.page_content for t in text_chunks]
    print(f"Number of texts: {len(texts)}")

    embeddings_list = embeddings.embed_documents(texts)
    print(f"Number of embeddings: {len(embeddings_list)}")

    # Prepare data for upsert
    vectors = [
        (str(i), embedding, {"text": text})
        for i, (embedding, text) in enumerate(zip(embeddings_list, texts))
    ]
    print(f"Number of vectors: {len(vectors)}")

    # Batch upsert to Pinecone
    batch_size = 100  # You can adjust this value
    for i in tqdm(range(0, len(vectors), batch_size)):
        batch = vectors[i:i+batch_size]
        index.upsert(vectors=batch)

    print("Data has been successfully loaded into Pinecone index.")

In [ ]:
pip install --upgrade pinecone-client langchain

In [ ]:
print(index.describe_index_stats())

In [ ]:
from langchain.vectorstores import Pinecone as LangchainPinecone
from pinecone import Pinecone
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize Pinecone
pc = Pinecone(api_key="e353333b-b041-4e67-888c-ea37f8b89dc5")

# Get the index
index = pc.Index(index_name)

# Initialize HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create a custom embedding function
def embed_query(query):
    return embeddings.embed_query(query)

# Create a Pinecone vector store for use with LangChain
docsearch = LangchainPinecone(
    index,
    embed_query,
    "text"  # This should match the metadata field name you used when uploading vectors
)

# Create a custom embedding function
def embed_query(query):
    return embeddings.embed_query(query)

query = "What is Literacy"

# Modify the similarity_search method to use keyword arguments
def modified_similarity_search(query, k=3, **kwargs):
    query_embedding = embed_query(query)
    results = index.query(vector=query_embedding, top_k=k, include_metadata=True, namespace='chatbot')
    
    documents = []
    for result in results['matches']:
        documents.append(result['metadata']['text'])
    
    return documents

# Use the modified similarity search
docs = modified_similarity_search(query, k=3)

print("Result:", docs)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="D:\End-to-end-Chatbot-using-Llama2\model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input = input(f"Input Prompt (type 'quit' to exit): ")
    if user_input.lower() == 'quit':
        print("Exiting the program...")
        break
    
    result = qa.invoke({"query": user_input})
    print("Response : ", result["result"])